In [3]:
!pip install google_auth_oauthlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [google_auth_oauthlib]


In [4]:
# Import necessary libraries
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from openai import OpenAI
from vllm import SamplingParams
from flask import Flask, request, jsonify
from threading import Thread
import json
from datetime import datetime, timedelta

INFO 07-12 10:32:24 [__init__.py:248] Automatically detected platform rocm.


In [5]:
# Authentication function
SCOPES = ["https://www.googleapis.com/auth/calendar"]

def authenticate(user):
    flow = InstalledAppFlow.from_client_secrets_file(f"Keys/{user.split('@')[0]}.amd.json", SCOPES)
    creds = flow.run_local_server(port=0)
    with open(f"Keys/{user.split('@')[0]}.amd.token", "w") as token:
        token.write(creds.to_json())
    return creds

In [11]:
def retrieve_calendar_events(user, start, end):
    events_list = []
    token_path = f"Keys/{user.split('@')[0]}.token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(
        calendarId='primary', timeMin=start, timeMax=end, singleEvents=True, orderBy='startTime'
    ).execute()
    events = events_result.get('items', [])
    
    for event in events:
        attendee_list = [attendee['email'] for attendee in event.get("attendees", [])] or ["SELF"]
        # Handle both dateTime and date keys
        start_time = event["start"].get("dateTime", event["start"].get("date"))
        end_time = event["end"].get("dateTime", event["end"].get("date"))
        events_list.append({
            "StartTime": start_time,
            "EndTime": end_time,
            "NumAttendees": len(set(attendee_list)),
            "Attendees": list(set(attendee_list)),
            "Summary": event.get("summary", "")
        })
    return events_list

In [7]:
# Configure vLLM server
BASE_URL = "http://localhost:3000/v1"
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"
client = OpenAI(api_key="NULL", base_url=BASE_URL, timeout=None, max_retries=0)

# AI Agent class
class AI_AGENT:
    def __init__(self, client, model_path):
        self.client = client
        self.model_path = model_path

    def parse_email(self, email_text):
        response = self.client.chat.completions.create(
            model=self.model_path,
            temperature=0.0,
            messages=[{
                "role": "user",
                "content": f"""
                You are an Agent that helps in scheduling meetings.
                Your job is to extract Email IDs and Meeting Duration.
                You should return:
                1. List of email IDs of participants (comma-separated).
                2. Meeting duration in minutes.
                3. Time constraints (e.g., 'next week').
                If the list of email IDs of participants are just names, append @amd.com at the end.
                Return as JSON with 'participants', 'time_constraints', and 'meeting_duration'.
                Strictly follow the instructions. Return only the dict with participants email IDs, time constraints, and meeting duration in minutes.
                Email: {email_text}
                """
            }]
        )
        return json.loads(response.choices[0].message.content)

# Initialize AI Agent
ai_agent = AI_AGENT(client, MODEL_PATH)

In [13]:
def your_meeting_assistant(data):
    # Parse email content
    meeting_details = ai_agent.parse_email(data["EmailContent"])
    duration = meeting_details["meeting_duration"]
    time_constraint = meeting_details["time_constraints"]
    
    # Determine start and end dates based on time constraint (e.g., next Thursday)
    today = datetime.now().replace(tzinfo=timezone.utc).astimezone(timezone(timedelta(hours=5, minutes=30)))
    if "thursday" in time_constraint.lower():
        days_ahead = (3 - today.weekday() + 7) % 7
        if days_ahead == 0:
            days_ahead = 7
        start_date = today + timedelta(days=days_ahead)
    else:
        start_date = today + timedelta(days=1)  # Default to next day for simplicity
    
    start_time = start_date.replace(hour=10, minute=30, second=0, microsecond=0).isoformat()
    end_time = (start_date + timedelta(minutes=int(duration))).replace(second=0, microsecond=0).isoformat()

    # Fetch events for attendees
    attendees = [data["From"]] + [attendee["email"] for attendee in data["Attendees"]]
    events = {email: retrieve_calendar_events(email, start_date.isoformat(), (start_date + timedelta(days=1)).isoformat()) for email in attendees}

    # Find a suitable time slot (simplified logic)
    proposed_event = {
        "StartTime": start_time,
        "EndTime": end_time,
        "NumAttendees": len(attendees),
        "Attendees": attendees,
        "Summary": data.get("Subject", "Meeting")  # Default to "Meeting" if Subject is missing
    }

    # Add events to attendees
    processed_attendees = [{"email": email, "events": events[email] + [proposed_event]} for email in attendees]

    # Prepare output
    output = {
        "Request_id": data["Request_id"],
        "Datetime": data["Datetime"],
        "Location": data.get("Location", ""),
        "From": data["From"],
        "Attendees": processed_attendees,
        "Subject": data.get("Subject", "Meeting"),  # Default to "Meeting" if Subject is missing
        "EmailContent": data["EmailContent"],
        "EventStart": start_time,
        "EventEnd": end_time,
        "Duration_mins": str(duration),
        "MetaData": {}
    }
    
    return {
        "processed": True,
        "output": output
    }

In [14]:
# Flask application
app = Flask(__name__)
received_data = []

@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    new_data = your_meeting_assistant(data)
    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data['output'], indent=2)}")
    return jsonify(new_data['output'])

# Run Flask in a background thread
def run_flask():
    app.run(host='0.0.0.0', port=5000)

Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [15]:
# Test case example
test_input = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "02-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [{"email": "usertwo.amd@gmail.com"}, {"email": "userthree.amd@gmail.com"}],
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

result = your_meeting_assistant(test_input)
print(json.dumps(result['output'], indent=2))

{
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "02-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-13",
          "EndTime": "2025-07-14",
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Weekend"
        },
        {
          "StartTime": "2025-07-13T18:00:00+05:30",
          "EndTime": "2025-07-14T09:00:00+05:30",
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-13T10:30:00+05:30",
          "EndTime": "2025-07-13T16:41:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com",
            "userthree.amd@gmail.com"
          ],
          "Summary